In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch.utils.data as Data

## Data Acquistion

### One-hot function

In [ ]:
def encode_onehot(labels): #生成独热码
    classes = set(labels)  # 注意因为 set ，所以每次生成的 y 的 onehot 值是不一样的
    classes_dict = {
        c: np.identity(len(classes))[i, :]
        for i, c in enumerate(classes)
    }  # np.identity() 创建一个单位对角阵， 单位矩阵的每一行对应一个one-hot向量
    labels_onehot = np.array(
        list(map(classes_dict.get, labels)),
        dtype=np.int32)  # map(function, iterable)： 对每个 label，应用 class_dict()
    return labels_onehot

### GetData Function

In [7]:
#Dataset-build dataset with data and labels
# 定义GetLoader类，继承Dataset方法，并重写__getitem__()和__len__()方法

class GetLoader(torch.utils.data.Dataset):
	# 初始化函数，得到数据
    def __init__(self, data_root, data_label):
        self.data = data_root
        self.label = data_label
        
    # index是根据batchsize划分数据后得到的索引，最后将data和对应的labels进行一起返回
    def __getitem__(self, index):
        data = self.data[index]
        labels = self.label[index]
        return data, labels #此前视频中和torchvision中采用的是targets
    
    # 该函数返回数据大小长度，目的是DataLoader方便划分，如果不知道大小，DataLoader会一脸懵逼
    def __len__(self):
        return len(self.data)

### Data Load

In [ ]:
# 随机生成数据，大小为10 * 20列
source_data = np.random.rand(10, 20)
# 随机生成标签，大小为10 * 1列
source_label = np.random.randint(0,2,(10, 1))
# 通过GetLoader将数据进行加载，返回Dataset对象，包含data和labels

#train_set
train_set = GetLoader(source_data, source_label)
#DataLoader
train_loader = Data.DataLoader(train_set, batch_size=32,num_workers=1,shuffle=True)

## Build Models

In [ ]:
class CNN(nn.Module):
    def __init__():
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=40, kernel_size=3)
        self.fc1 = nn.Linear(in_channels=SIZE,out_channels=60)
        self.fc2 = nn.Linear(in_channels=60, out_channels=30)
        self.out = nn.Linear(in_channels=30, out_channels=2)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.fc1(x.reshape(-1, SIZE)))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

## Instance the model

In [ ]:
network = CNN()
optimizer = optim.Adam(network.parameters(),lr=0.01)
train_loader = Data.DataLoader(train_set, batch_size=32,num_workers=1,shuffle=True)
batch = next(iter(train_loader))

## Training Process

In [8]:
epoch_max = 5
epoch = 0
for epoch in range(epoch_max):
    total_loss = 0
    total_correct = 0
    
    for batch in train_loader:
        data, labels = batch
        preds = network(data)
        loss = F.cross_entropy(preds, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #————————————————————————————————————————————————#
        #test
        if step%50 ==0:
            test_output = CNN(test_x)
            pred_y = torch.max(test_output,1)[1].data.squeeze()
            accuracy = float(sum(pred_y ==test_y))/float(test_y.size(0))
            print('Epoch:',epoch,'|train_loss:%.4f'%loss.data.numpy(),'|test accuracy:%.2f'%accuracy)
        
        
        total_correct += preds.argmax(dim=1).eq(labels).sum().item()
    print('epoch:',epoch,'|total_correct:',total_correct)

train_accuracy = total_correct/len(train_set)
train_accuracy

RuntimeError: DataLoader worker (pid(s) 7920) exited unexpectedly